# Probemos Jupyter Notebooks

Importo csv a Py usando la libreria panda:


In [35]:
import pandas as pd
import altair as alt

ruta_archivo = "C:/Documentos/n76359/Maestria/Python/pruebaGraf/dataset.csv"
datos = pd.read_csv(ruta_archivo, sep=';')

Reviso la base:

In [36]:
for data in datos:
    print(data)
    print(type(data))

TipoOperacion
<class 'str'>
Comercial/Financiero
<class 'str'>
Detalle
<class 'str'>
Periodo
<class 'str'>
Monto
<class 'str'>


Conviero columna Monto a número y creo base en millones de $:

In [37]:
datos["Monto"] = pd.to_numeric(datos["Monto"], errors="coerce")
datos_en_miles = datos
datos_en_miles["Monto"] = datos_en_miles["Monto"] / 1000000



In [51]:

Monto_agrupado = datos_en_miles.groupby(["Periodo", "TipoOperacion"])["Monto"].sum().reset_index()


min_sum_Monto = Monto_agrupado["Monto"].min()
max_sum_Monto = Monto_agrupado["Monto"].max()



Grafico = alt.layer(
    alt.Chart(
        datos
    ).mark_line(
        interpolate="monotone"    
    ).encode(
        
        x=alt.X("Periodo:N",
                axis=alt.Axis(
                    labelAngle=0,
                    grid=True,
                    gridColor="lightgray",
                    title="Periodo")
        ),
        y=alt.Y("Monto:Q", 
                aggregate="sum",
                axis=alt.Axis(
                    title="Monto en millones de $" , 
                    titlePadding=10,
                    
                    grid=True,
                    gridColor="lightgray",
                    
                    titleOpacity=0.70), 
                scale=alt.Scale(domain=[min_sum_Monto*0.99 , 
                                max_sum_Monto *1.01])
        ),    
        color="TipoOperacion"),
    # Puntos
    alt.Chart(Monto_agrupado).mark_point(size=80).encode(
        x="Periodo:N",
        y=alt.Y("Monto:Q", aggregate="sum"),
        color="TipoOperacion"
    ).transform_filter(
        f"datum.Monto == {max_sum_Monto} || datum.Monto == {min_sum_Monto}"  # Filtrar por máximos y mínimos
    )
).properties(
    width=650,
).configure(
    background="lightyellow" 
)




In [39]:
Grafico.show()

alt.LayerChart(...)

In [74]:
selector = alt.selection_point(fields=["Periodo"], bind="legend")  # Selección basada en la leyenda


Grafico2 = alt.Chart(datos_en_miles).transform_filter(
    selector  # Filtrar valores basados en la selección
    ).mark_bar().encode(
    x=alt.X("Monto:Q", axis=alt.Axis(grid=False)),
    y=alt.Y("Detalle:O",sort="-x", axis=alt.Axis(labelLimit=7000)),
    color=alt.Color("Periodo:N", legend=alt.Legend(title="Selecciona Periodo")),
    opacity=alt.condition(selector, alt.value(1), alt.value(0.2))  # Opacidad basada en la selección

    ).add_params(
        selector  # Agregar la selección al gráfico
    ).properties(
        width=800,
        height=600
    ).configure(
        background="lightyellow" 
    )

Grafico2.show()

alt.Chart(...)